# Table of Contents
 <p>

In [13]:
import pandas as pd
from openpyxl import load_workbook
import datetime

In [14]:
#解决字符编码问题
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [15]:
header=["ticket_record_id","begin_station_id","end_station_id","unit_price","tk_num","userid","original_type",
        "pay_type","target","description","trade_no","tal_fee","time_end","voucher","err_status","pay_result_code",
        "notify_type","finish_num","notify_time","refund_type","result_code","refund_fee","refund_time","error_code",
        "error_detail","order_status","ticketurl","create_time","ticket_no","station_code","product_code"]
data=pd.read_csv('./数据（1-6月）/APPdata/u_user_ticket_record_his.csv')
data.head()

,ticket_record_id,begin_station_id,end_station_id,unit_price,tk_num,userid,original_type,pay_type,target,description,...,refund_fee,refund_time,error_code,error_detail,order_status,ticketurl,create_time,ticket_no,station_code,product_code
0,0000086a-025f-11e7-bc41-005056900b11,408.0,314.0,500,1,NaN,4,NaN,TVM,NaN,...,0,0.0,NaN,NaN,2,site_begin=0408&site_end=0314&ticket_num=1&tic...,2017/3/6 19:12:52,78521027861444077184,NaN,NaN
1,00001488-2d89-11e7-bc41-005056900b11,825.0,410.0,300,1,e7e9f4d8-e6a9-4b65-9e7b-b7a088652239,4,NaN,TVM,NaN,...,0,0.0,NaN,NaN,2,site_begin=0825&site_end=0410&ticket_num=1&tic...,2017/4/30 17:34:35,12343063643511062005,NaN,NaN
2,000015dd-f1c6-11e6-ab56-005056900b11,314.0,622.0,200,1,NaN,4,NaN,TVM,NaN,...,0,0.0,NaN,NaN,2,site_begin=0314&site_end=0622&ticket_num=1&tic...,2017/2/13 16:30:47,59853988406836477639,NaN,NaN
3,000017f4-1f57-11e7-bc41-005056900b11,105.0,245.0,400,1,40d9f40e-65a5-42f6-8b75-46782c3143f2,3,NaN,TVM,NaN,...,0,0.0,NaN,NaN,2,site_begin=0105&site_end=0245&ticket_num=1&tic...,2017/4/12 16:06:39,12343282726191495265,NaN,NaN
4,00002d20-0403-11e7-bc41-005056900b11,302.0,3004.0,600,1,NaN,4,NaN,TVM,NaN,...,0,0.0,NaN,NaN,2,site_begin=0302&site_end=3004&ticket_num=1&tic...,2017/3/8 21:26:48,46721183805617024225,NaN,NaN


In [16]:
station=pd.read_csv('./数据（1-6月）/APPdata/ticket_station.csv')
station.head()

,id,stationid,cn_name,en_name
0,1,101,西朗,Xilang
1,2,102,坑口,Kengkou
2,3,103,花地湾,Huadiwan
3,4,104,芳村,Fangcun
4,5,105,黄沙,Huangsha


In [17]:
#数据格式处理
station['stationid']=station['stationid'].apply(float)
data['time_end']=pd.to_datetime(data['time_end'],format='%Y%m%d%H%M%S')
# print (data['time_end'])
data=data[data['result_code'] == 'SUCCESS']
data=data[data['order_status'] == 2]
station = station.drop_duplicates('stationid')

In [18]:
#添加开始／结束站点
# def add_station(a):
#     for j in range(len(station)):
#         if (a==station['stationid'][j]):
#             return station['cn_name'][j]
        
# data['begin_station']=data['begin_station_id'].apply(add_station)
# data['end_station']=data['end_station_id'].apply(add_station)

data = data.merge(station,left_on=['begin_station_id'],right_on=['stationid'],how='left')
data = data.rename(columns={'cn_name':'begin_station'})
data = data.merge(station,left_on=['end_station_id'],right_on=['stationid'],how='left')
data = data.rename(columns={'cn_name':'end_station'})

In [19]:
#excel写入函数
def excelAddSheet(dataframe,excelWriter,sheetname):
   book = load_workbook(excelWriter.path)
   excelWriter.book = book
   dataframe.to_excel(excel_writer=excelWriter,sheet_name=sheetname,index=None)
   excelWriter.close()

excelPath='./数据（1-6月）/APPdata/hot_station.xlsx'
excelWriter=pd.ExcelWriter(excelPath,engine='openpyxl')

In [20]:
#筛选热门起终点站
# print (data['begin_station_id'].value_counts())
# print (data['begin_station_id'].value_counts().mean())
hot_begin_station=data['begin_station'].value_counts()[data['begin_station'].value_counts() > data['begin_station_id'].value_counts().mean()]
# print (hot_begin_station.index[:10])
hot_begin_station=pd.DataFrame(hot_begin_station,index=hot_begin_station.index)
hot_begin_station['num']=range(len(hot_begin_station))
hot_begin_station['hot_begin_station']=hot_begin_station.index
hot_begin_station.index=hot_begin_station['num']
del hot_begin_station['num']
hot_begin_station.columns=[u'购票数量',u'站点']
# print (hot_begin_station)
excelAddSheet(hot_begin_station,excelWriter,'hot_begin_station')

# print (data['end_station_id'].value_counts())
# print (data['end_station_id'].value_counts().mean())
hot_end_station=data['end_station'].value_counts()[data['end_station'].value_counts() > data['end_station_id'].value_counts().mean()]
# print (hot_end_station.index[:10])
hot_end_station=pd.DataFrame(hot_end_station,index=hot_end_station.index)
hot_end_station['num']=range(len(hot_end_station))
hot_end_station['hot_begin_station']=hot_end_station.index
hot_end_station.index=hot_end_station['num']
del hot_end_station['num']
hot_end_station.columns=[u'购票数量',u'站点']
# print (hot_end_station)
excelAddSheet(hot_end_station,excelWriter,'hot_end_station')

In [21]:
#热门起终点站客流走势分析
data['week_day']=data['time_end'].apply(lambda x: x.dayofweek)
data['time_end_day']=data['time_end'].apply(pd.Timestamp.date)

#广州南站
# print (data['time_end_day'][data['begin_station_id'] == 249.0].value_counts().sort_index())
# print (data['time_end_day'][data['end_station_id'] == 249.0].value_counts().sort_index())
print (data['time_end_day'][(data['begin_station_id'] == 249.0) & 
                            ((data['week_day'] == 0) | (data['week_day'] == 6))].value_counts().mean())
# print (data['time_end_day'][(data['begin_station_id'] == 249.0) & 
#                             ((data['week_day'] == 0) | (data['week_day'] == 6))].value_counts().sort_index())
print (data['time_end_day'][(data['begin_station_id'] == 249.0) & 
                            (data['week_day'] != 0) & (data['week_day'] != 6)].value_counts().mean())
# print (data['time_end_day'][(data['begin_station_id'] == 114.0) & 
#                             (data['week_day'] != 0) & (data['week_day'] != 6)].value_counts().sort_index())

#体育西路
# print (data['time_end_day'][data['begin_station_id'] == 114.0].value_counts().sort_index())
# print (data['time_end_day'][data['end_station_id'] == 114.0].value_counts().sort_index())
print (data['time_end_day'][(data['begin_station_id'] == 114.0) & 
                            ((data['week_day'] == 0) | (data['week_day'] == 6))].value_counts().mean())
# print (data['time_end_day'][(data['begin_station_id'] == 114.0) & 
#                             ((data['week_day'] == 0) | (data['week_day'] == 6))].value_counts().sort_index())
print (data['time_end_day'][(data['begin_station_id'] == 114.0) & 
                            (data['week_day'] != 0) & (data['week_day'] != 6)].value_counts().mean())
# print (data['time_end_day'][(data['begin_station_id'] == 114.0) & 
#                             (data['week_day'] != 0) & (data['week_day'] != 6)].value_counts().sort_index())

# print(hot_begin_station[u'站点ID'])

#热门起点站
with pd.ExcelWriter('./数据（1-6月）/APPdata/hot_begin_station_people.xlsx') as writer:
    for station in hot_begin_station[u'站点']:
        temp=data['time_end_day'][data['begin_station'] == station].value_counts().sort_index()
        temp.name=u'购票量'
        temp.to_excel(writer, sheet_name=str(station))

#热门终点站
with pd.ExcelWriter('./数据（1-6月）/APPdata/hot_end_station_people.xlsx') as writer:
    for station in hot_end_station[u'站点']:
        temp=data['time_end_day'][data['end_station'] == station].value_counts().sort_index()
        temp.name=u'购票量'
        temp.to_excel(writer, sheet_name=str(station))
        
#周末时期
with pd.ExcelWriter('./数据（1-6月）/APPdata/hot_station_weekend.xlsx') as writer:
    for station in hot_end_station[u'站点']:
        temp=data['time_end_day'][(data['begin_station'] == station) & 
                                  (data['week_day'] == 0) | (data['week_day'] == 6)].value_counts().sort_index()
        temp.name=u'购票量'
        temp.to_excel(writer, sheet_name=str(station))

In [33]:
#热门站点进站高峰期分析
data['time_end_hour']=data['time_end'].apply(lambda x: x.hour)

#广州南站
print (data['time_end_hour'][data['begin_station_id'] == 249.0].value_counts().sort_index())

#体育西路
print (data['time_end_hour'][data['begin_station_id'] == 114.0].value_counts().sort_index())

#广州塔
print (data['time_end_hour'][data['begin_station_id'] == 305.0].value_counts().sort_index())


with pd.ExcelWriter('./数据（1-6月）/APPdata/hot_station_hour.xlsx') as writer:
    for station in hot_begin_station[u'站点']:
        temp=data['time_end_hour'][data['begin_station'] == station].value_counts().sort_index()
        temp.name=u'购票量'
        temp.to_excel(writer, sheet_name=str(station))
        
with pd.ExcelWriter('./数据（1-6月）/APPdata/test.xlsx') as writer:
    temp1=data['time_end_hour'][(data['begin_station_id'] == 249.0) & 
                               (data['time_end_day'] == datetime.date(2017,5,1))].value_counts().sort_index()
    temp2=data['time_end_hour'][(data['begin_station_id'] == 249.0) & 
                               (data['time_end_day'] == datetime.date(2017,3,15))].value_counts().sort_index()
    temp=pd.concat([temp1,temp2])
    temp.to_excel(writer, sheet_name=str(249))
    
with pd.ExcelWriter('./数据（1-6月）/APPdata/test.xlsx') as writer:
    temp1=data['time_end_hour'][((data['week_day'] == 0) | (data['week_day'] == 6))].value_counts().sort_index()
    temp2=data['time_end_hour'][((data['week_day'] != 0) & (data['week_day'] != 6))].value_counts().sort_index()
    temp=pd.concat([temp1,temp2])
    temp.to_excel(writer, sheet_name=str(249))

In [32]:
#test
data['time_end_day'][1]

datetime.date(2017, 4, 30)

In [27]:
#热门线路统计
#按日期
hot_route=data.loc[:,['time_end_day','begin_station','end_station','tk_num']]
hot_route=hot_route.groupby(['time_end_day','begin_station','end_station'],as_index=False).sum()
# print (hot_route.loc[:,['begin_station','end_station','tk_num']][hot_route['time_end_day'] == pd.Timestamp('2017-05-01').date()].sort_values(by=['tk_num'],ascending=False))

with pd.ExcelWriter('./数据（1-6月）/APPdata/hot_route.xlsx') as writer:
    for day in hot_route['time_end_day'].unique():
        temp = hot_route.loc[:,['begin_station','end_station','tk_num']][hot_route['time_end_day'] == day].sort_values(by=['tk_num'],ascending=False)
        temp.to_excel(writer, sheet_name=str(day))
        
#总量
hot_route=data.loc[:,['begin_station','end_station','tk_num']]
hot_route=hot_route.groupby(['begin_station','end_station']).sum()
# print (hot_route)